# 循环神经网络
RNN是一种非常通用的神经网络，无论是图像识别还是声音识别，文字识别，NLP时间序列的数据，周期的数据等都是通用合适的。

在传统的神经网络模型中，是从输入层到输出层，层与层之间是全连接的，每层之间的节点是无连接的。但是这种普通的神经网络对于很多问题却无能为力。例如，你要预测句子的下一个单词是什么，一般需要用到前面的单词，因为一个句子中前后单词并不是独立的。RNNs之所以称为循环神经网路，即一个序列当前的输出与前面的输出也有关。具体的表现形式为网络会对前面的信息进行记忆并应用于当前输出的计算中，即隐藏层之间的节点不再无连接而是有连接的，并且隐藏层的输入不仅包括输入层的输出还包括上一时刻隐藏层的输出。理论上，RNNs能够对任何长度的序列数据进行处理。但是在实践中，为了降低复杂性往往假设当前的状态只与前面的几个状态相关.比如 图片 如果28x28像素 如果我们把每一行像素跟上一行的像素 看做时间系列处理的话，也就是 在传统的神经网络WX+B 引入了t 每一个时间系列 t范围内，下一次的w 通过上一个t-1 的wx+b得到 w 所以 图像如果使用RNN处理适合 图像理解 效果很好，NLP 效果也很好。

![](http://static.oschina.net/uploads/space/2016/0424/160913_kyzR_937000.png)

![](http://static.oschina.net/uploads/space/2016/0424/161016_oxQ0_937000.jpg)

下面看看RNN输入， 首先seq/step 时间系列 ，如果28x28的图像 RNN处理的话，其实就是每一个seq/step=28长度 28的输入，所以 对于tensorflow 来说输入维度[size_batches, seq_length, rnn_size]

下面 我们看看 输入的X为【size_batches,input] 要转换成上面的格式 所以需要reshape 重新维度

 xdata.reshape((batch_size, n_steps, n_input)) 转换成 [batch,28,28] 其实28个t 上下文计算  最后的t得到输出的w b 用于全连接。也就说28步 step
 
 

在tensorflow中，用于颠倒交换维度的

```
_x = tf.transpose(_X, [1, 0, 2] )
```

上面 对于3维的X 默认是 0 1 2 系数 ，我们把它置换成1 0 2 也就是 第一维 第二维进行交换
比如  (batch_size, n_steps, n_input) 交换后就是 （nsteps,batch_size,n_input) 这正好就是我们上面需要的28步step的
（batch_size,n_input）的输入。但是这是三维的数组 所以需要降维重新展开reshape 
tf.reshape(_X, [-1, n_input])  这里我讲一下 reshape 就是重新展开 定义维度。-1 表示站位，比如 一个多维的数组，[2 ,2 ,3] 的维度的3维数组，reshape(【-1,3】)  表示要变成 后面维度为3 那么前面的-1 表示全部展开后，除掉 3 那么其实就是shape(4,3) 的维度 二维数组。

所以上面的我们只需要reshape([-1, n_input])  那么 此时将会的到一个(n_steps*batch_size, n_input)

下面的就遵循矩阵运算 WX+B一步一步即可 最后每一个28step 的每一个step得到一个outpput 根据上下文 上文计算下文的w 所以 最后的输出 就是最后一个元素得到的w 将会最后进行全连接连接层wx+b。

下面看看官方案例的一个例子 。 此时 把图像 从 28x28 当做一个 28 step的  28 input的 RNN ，就好像记忆一样，我们现在所有的决定 都是过去的经验经历 记忆所影响的。 后面有一个RNN上 的变体，LSTM 长短期记忆网络 解决了RNN的缺点 ，对RNN隐藏层 进行改进

这里我们把RNN/LSTM 放在一起 ，是因为他本质是一样的 。

LSTM 就是把RNN的单元 换了更好的单元 就是加上了记忆。所以LSTM 长短期记忆网络

长期是通过遗忘门进行调节。 短期是通过记忆门进行调节。

## LSTM引入了Cell 与其说LSTM是一种RNN结构，倒不如说LSTM是RNN的一个魔改组件，把上面看到的网络中的小圆圈换成下面的LSTM的结构

![](http://static.oschina.net/uploads/space/2016/0429/040637_SxV1_937000.jpg)

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [16]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np

In [6]:
# Parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

In [7]:
# Network Parameters
n_input = 28 # MNIST data input (img shape: 28*28)
n_steps = 28 # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 10 # MNIST total classes (0-9 digits)

In [8]:
# tf graph
x = tf.placeholder(tf.float32, [None, n_steps, n_input])
istate = tf.placeholder(tf.float32, [None, 2*n_hidden])
y = tf.placeholder(tf.float32, [None, n_classse])

In [9]:
# define weights
weights = {
    'hidden': tf.Variable(tf.random_normal([n_input, n_hidden])), 
    'out': tf.Variable(tf.random_normal([n_hidden, n_classse]))
}
biases = {
    'hidden': tf.Variable(tf.random_normal([n_hidden])),
    'out': tf.Variable(tf.random_normal([n_classse]))
}

In [21]:
def RNN(_X, _istate, _weights, _biases):
    _X = tf.transpose(_X, [1, 0, 2])
    _X = tf.reshape(_X, [-1, n_input])
    _X = tf.matmul(_X, _weights['hidden']) + _biases['hidden']
    
    lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    _X = tf.split(_X, n_steps, 0)
    
    
    outputs, states = rnn.RNNCell(lstm_cell, _X, _istate)
    
    return tf.matmul(outputs[-1], _weights['out']) + _biases['out']

In [22]:
pred = RNN(x, istate, weights, biases)

TypeError: Cannot convert Tensor("Placeholder_1:0", shape=(?, 256), dtype=float32) to a dtype. data type not understood